In [1]:

import pandas as pd 
from mlbase.db import MLData, Table
from functools import partial
from pyatlasapi.visualising import upload_layer
from datetime import datetime
from docxcompose.composer import Composer
from docx import Document as Document_compose
import os
from collections import Counter
import numpy as np


In [3]:
last_date = '2021-06-01'
future_date = '2021-07-01'

In [4]:
def no_sales_marker(row, tabs):
    balance = [
        f'Остатки {tabs}'
    ]
    sale = [
        f'Продажи {tabs}'
    ]
    
    return 1 * (row[balance].gt(0).all() and row[sale].sum() <= 0)


def lpu_visits_color(row):
    if row["Количество врачей 'В норме'"] == row['Число всех врачей']:
        return 'green'
    else:
        return 'red'
    
    
def lpu_key_color(row):
    if row['Ключевое ЛПУ']:
        return 'green'
    else:
        return 'red'
    
    
def lpu_attent_color(row):
    if row['Ключевое ЛПУ']:
        return lpu_visits_color(row)
    else:
        return 'black'
    

def spec_visits_color(row):
    if row['Посещения в норме']:
        return 'green'
    else:
        return 'red'
    
    
def spec_key_color(row):
    if row['Ключевое ЛПУ']:
        return 'green'
    else:
        return 'red'
    
    
def spec_attent_color(row):
    if row['Ключевое ЛПУ']:
        return spec_visits_color(row)
    else:
        return 'black'


def stores_colors(row):
    if row['Флаг внимания'] == '':
        return 'black'
    else:
        check = [
            'Остатки 28(50)',
            'Остатки 56(50)',
            'Остатки 28(100)',
            'Остатки 56(100)',
            'Остатки 28(200)',
            'Остатки 56(200)'
        ]
        if row[check].sum() == 0:
            return 'red'
        else:
            return 'green'

In [5]:
def df_creator(last_date, future_date):
    year = pd.to_datetime(last_date).year
    last_month = pd.to_datetime(last_date).month
    future_month = pd.to_datetime(future_date).month
    
    

    sale_balance_names = [
        f'{what}_{tablet}_{dose}'
        for what in ('sale', 'balance')
        for dose in (50, 100, 200)
        for tablet in (28, 56)
    ]
    sale_balance_sql = [
        f'''
            SELECT
                point_id,
                "{what[0]} {tablet}({dose})"

            FROM
                (
                SELECT 
                    point_id, 
                    date,
                    value as "{what[0]} {tablet}({dose})"

                FROM 
                    (
                    SELECT
                        *
                    FROM
                        
                        drug_stores_new_target
                    WHERE 
                        target_type='{what[1]}' 
                        and dose={dose} 
                        and tablets={tablet}
                    ) a
                WINDOW w as (
                    PARTITION BY point_id
                    ORDER BY point_id, date DESC
                    )
                ) b
            WHERE date = '{last_date}' 
            '''
        for what in (
            ('Продажи', 'sale'), 
            ('Остатки', 'balance')
        )
        for dose in (50, 100, 200)
        for tablet in  (28, 56)
    ]
    
    

    
    
    sale_balance_table_expr = ', '.join([f'{name} as ({sql})' for name, sql in zip(sale_balance_names, sale_balance_sql)])

    sale_balance_names_g = [
        f'{what}_{tablet}'
        for what in ('sale', 'balance')
        for tablet in (28, 56)
    ]
    

    
    
    sale_balance_sql_g = [
        f'''
        SELECT
            point_id,
            "{what[0]} {tablets}",
            "Накопленный итог, {what[0]} #{tablets}"

        FROM 
        (SELECT 
                    point_id, 
                    date,
                    "{what[0]} {tablets}",
                    sum("{what[0]} {tablets}") over (partition by point_id order by date
                        rows between unbounded preceding and current row)
                        as "Накопленный итог, {what[0]} #{tablets}"
                        
                FROM (
                    
                    
            SELECT 
                point_id, 
                date,
                sum(value) as "{what[0]} {tablets}"
                
            FROM 
                (
                SELECT
                    point_id,
                    date,
                    value
                FROM 
                    drug_stores_new_target
                WHERE 
                    target_type='{what[1]}' 
                    and tablets={tablets}
                ) a
            GROUP BY point_id, date
                
        ) b
        ) с
        
            WHERE date = '{last_date}'
        '''
        for what in (
            ('Продажи', 'sale'), 
            ('Остатки', 'balance')
        )
        for tablets in (28, 56)
    ]
    sale_balance_table_expr_g = ', '.join([f'{name} as ({sql})' for name, sql in zip(sale_balance_names_g, sale_balance_sql_g)])

    pred_names = [
        f'prediction_{dose}_{tablets}'
        for dose in (50, 100, 200)
        for tablets in (28, 56)
    ]
    pred_sql = [
        f'''SELECT point_id, 
            prediction "Прогноз на следующий месяц {tablets}({dose})", 
            6*prediction "Прогноз на пол года {tablets}({dose})"
            FROM drug_stores_prediction
            WHERE 
                date = '{future_date}'
                AND dose = {dose}
                AND tablet = {tablets}
            '''
        for dose in (50, 100, 200)
        for tablets in (28, 56)
    ]
    pred_expressions = ', '.join([f'{name} as ({sql})' for name, sql in zip(pred_names, pred_sql)])

    pred_name_gr = [f'prediction_sum_{tablets}' for tablets in (28, 56)]
    pred_expr_gra=[
        f'''
            SELECT 
                point_id, 
                sum(prediction) "Прогноз на следующий месяц {tablets}", 
                6*sum(prediction) "Прогноз на полгода {tablets}"
            FROM drug_stores_prediction
        WHERE
            date = '{future_date}'
        AND tablet = {tablets}
        GROUP BY
            point_id
            '''
        for tablets in (28, 56)]
    
    pred_expr_gr = ', '.join([f'{name} as ({sql})' for name, sql in zip(pred_name_gr, pred_expr_gra)])
    

    expr = f'{sale_balance_table_expr}, {sale_balance_table_expr_g}, {pred_expressions}, {pred_expr_gr}'

    joins = ' '.join([f'LEFT JOIN {name} USING(point_id)' 
                      for name in sale_balance_names + sale_balance_names_g + pred_names + pred_name_gr])
    

    stores_sql = f'''
    with visits_new as (
        SELECT
            index,
            string_agg(distinct "Territory", ', ') as ter,
            count(distinct "Account: External ID") as specs
        FROM
            lpu_mr_connection
        WHERE
                "Account: Primary Specialty" = 'Cardiology'
        GROUP BY
            index
    ),
    visits_agg as(
        SELECT
            index, 
            sum(visits) as visits,
            sum(specialists) as specialists
        FROM
            lpu_visits_agg 
        WHERE
            "Account: Primary Specialty" = 'Cardiology'
            AND "Call Method" = 'Face to Face'
            AND month = {last_month}
            AND year = {year}
        GROUP BY
            index
    ),
    stores as (
        SELECT
            point_id,
            net as "Аптечная сеть",
            address "Адрес", 
            gc_lat "Широта",
            gc_lng "Долгота",
            gc_state "Регион",
            gc_city "Город"
        FROM
            drug_stores_base
        WHERE
            ends_at = '2021-09-01'
    ), {expr}, lpu_info as (
        SELECT
            dsb.point_id,
            lns.distance "Расстояние до ближнего ЛПУ", 
            lbn.name "Название ЛПУ", 
            lbn.gc_state "Регион ЛПУ",
            lbn.gc_city "Город ЛПУ",
            lbn.gc_address "Адрес ЛПУ",
            lbn.gc_lat "Широта ЛПУ",
            lbn.gc_lng "Долгота ЛПУ",
            v.ter "Территория",
            v.specs "Число посещ. врачей в ближнем ЛПУ",
            va.visits "Число визитов в ближнее ЛПУ",
            va.specialists "Число врачей в ближнем ЛПУ"
        FROM
            drug_stores_base dsb

            LEFT JOIN (
                SELECT
                    point_id store_id,
                    distance,
                    lpu_id
                FROM
                    drug_stores_base ds,
                    LATERAL (
                        SELECT
                            index lpu_id,
                            ST_Distance(l.geo, ds.geography) distance
                        FROM
                            lpu_base_new l
                        WHERE
                            ST_DWithin(l.geo, ds.geography, 4000)
                            AND not is_drug_store
                        ORDER BY
                            l.geo <-> ds.geography ASC
                        LIMIT 1
                    ) joined
                    
            ) lns on lns.store_id = dsb.point_id  
            LEFT JOIN lpu_base_new lbn on lns.lpu_id = lbn.index 
            LEFT JOIN visits_new v USING(index)
            LEFT JOIN visits_agg va USING(index)
    ), geo_info as (
        SELECT
            point_id,
            "Квартиры, балл",
            "общественный транспорт, балл",
            "Трафик, балл",
            "Парковка, балл",
            "Детский сад, балл",
            "Итоговый балл"
        FROM
            drug_stores_base_geo_joined
    ), first_sale as (
        SELECT
            DISTINCT ON (point_id)
            point_id,
            concat(extract(month from date::date), '`', to_char(date::date, 'YY')) as "Месяц первых продаж"
        FROM
            drug_stores_sale_new
        WHERE
            value > 0
        ORDER BY
            point_id ASC,
            date ASC
    ), kams as (
        SELECT 
            point_id,
            "КАМ"
            from drug_stores_base
            LEFT JOIN kams
            USING(net)
            
        ),
        
        
        nearest_lpus as
        (
        SELECT point_id,
                sum(visits) "Число визитов в ЛПУ в радиусе 1 км",
                sum(specialists) "Число посещ. врачей в радиусе 1 км"
            FROM
        
        (SELECT 
                point_id, 
                lpu_id,
                visits,
                specialists
                
            FROM
                    drug_stores_base ds,
                    LATERAL (
                        SELECT
                            index lpu_id
                        FROM
                            lpu_base_new l
                        WHERE
                            ST_DWithin(l.geo, ds.geography, 1000)
                            AND not is_drug_store
        
        ) nl
        LEFT JOIN
        LATERAL (
        SELECT 
                index,
                sum(visits) visits, 
                sum(specialists) specialists
        FROM 
                lpu_visits_agg
        WHERE 
                "Account: Primary Specialty" = 'Cardiology'
            AND
                "Call Method" = 'Face to Face'
            AND month = {last_month}
            AND year = {year}
            AND visits is not NULL
            AND specialists is not NULL
        GROUP BY
            index
                )
         lva
        on nl.lpu_id=lva.index
        ) nlp
        
        GROUP BY 
                    point_id),
                    
                    
         ns as (           
        SELECT point_id, 
                count(distinct lpu_id) "Число ЛПУ в радиусе 1 км",
                sum(specialists) "Число врачей в ЛПУ в радиусе 1 км"
            FROM
        
        (SELECT 
                point_id, 
                lpu_id,
                specialists
                
            FROM
                    drug_stores_base ds,
                    LATERAL (
                        SELECT
                            index lpu_id
                        FROM
                            lpu_base_new l
                        WHERE
                            ST_DWithin(l.geo, ds.geography, 1000)
                            AND not is_drug_store
        
        ) nl
        LEFT JOIN
        LATERAL (
        SELECT 
                index,
                count(distinct "Account: External ID") specialists
        FROM 
                lpu_mr_connection
                
        WHERE
                "Account: Primary Specialty" = 'Cardiology'

        GROUP BY
            index
                )
         lva
        on nl.lpu_id=lva.index
        ) nlp
        
        GROUP BY 
                    point_id
        )
        
    
        
        
        

        
    SELECT 
        CASE 
            WHEN "Итоговый балл" >= 6 AND coalesce("Расстояние до ближнего ЛПУ", 5000) <= 1000
            THEN 1
        ELSE
            0
        END "Флаг внимания", '{last_date}' as "Дата",
        extract(year from '{last_date}'::date) as "Год", 
        concat('Q', extract(quarter from '{last_date}'::date),'`', to_char('{last_date}'::date, 'YY')) "Квартал", 
        concat(extract(month from '{last_date}'::date), '`', to_char('{last_date}'::date, 'YY')) as "Месяц",
        
        * 
    FROM
        stores
        {joins}
        LEFT JOIN lpu_info USING(point_id)
        LEFT JOIN geo_info USING(point_id)
        LEFT JOIN first_sale USING(point_id)
        LEFT JOIN kams USING(point_id)
        LEFT JOIN nearest_lpus USING(point_id)
        LEFT JOIN ns USING(point_id)
    '''
    

    df = pd.read_sql(stores_sql, db.engine)

    for tabs in 28, 56:
        df[f'Нет продаж при наличии остатков {tabs}'] = df.apply(no_sales_marker, axis=1, tabs=tabs)
    
    

    order = [
        'point_id', 
        'Дата',
        'Год',
        'Квартал',
        'Месяц',
        'Адрес', 
        'Регион',
        'Город',
        'Широта',
        'Долгота',
        'КАМ',
        'Аптечная сеть', 
        'Месяц первых продаж',
        
        "Накопленный итог, Продажи #28",
        "Накопленный итог, Продажи #56",
        

        'Название ЛПУ',
        "Регион ЛПУ",
        "Город ЛПУ",
        "Адрес ЛПУ",
        "Широта ЛПУ",
        "Долгота ЛПУ",
        'Расстояние до ближнего ЛПУ', 
        'Территория', 
        'Число врачей в ближнем ЛПУ',
        'Число посещ. врачей в ближнем ЛПУ',
        'Число визитов в ближнее ЛПУ',
        "Число врачей в ЛПУ в радиусе 1 км",
        "Число посещ. врачей в радиусе 1 км",
         "Число визитов в ЛПУ в радиусе 1 км",
        "Число ЛПУ в радиусе 1 км",

        'Флаг внимания',
        'Итоговый балл',
        'Квартиры, балл',
        'общественный транспорт, балл', 
        'Трафик, балл', 
        'Парковка, балл', 
        'Детский сад, балл', 

        'Нет продаж при наличии остатков 28',
        'Нет продаж при наличии остатков 56',


        'Продажи 28',
        'Продажи 56',

        'Продажи 28(50)',
        'Продажи 28(100)',
        'Продажи 28(200)',

        'Продажи 56(50)',
        'Продажи 56(100)',
        'Продажи 56(200)',


        'Остатки 28',
        'Остатки 56',

        'Остатки 28(50)',
        'Остатки 28(100)',
        'Остатки 28(200)',

        'Остатки 56(50)',
        'Остатки 56(100)',
        'Остатки 56(200)',
 
        'Прогноз на следующий месяц 28(50)', 
        'Прогноз на пол года 28(50)', 
        'Прогноз на следующий месяц 28(100)', 
        'Прогноз на пол года 28(100)', 
        'Прогноз на следующий месяц 28(200)', 
        'Прогноз на пол года 28(200)', 
        'Прогноз на следующий месяц 28', 
        'Прогноз на полгода 28',
        
        'Прогноз на следующий месяц 56(50)', 
        'Прогноз на пол года 56(50)', 
        'Прогноз на следующий месяц 56(100)', 
        'Прогноз на пол года 56(100)', 
        'Прогноз на следующий месяц 56(200)', 
        'Прогноз на пол года 56(200)', 
        'Прогноз на следующий месяц 56', 
        'Прогноз на полгода 56'


    ]

    df = df[order]

    df[[x for x in df if 'прогноз на' in x.lower()]] = df[[x for x in df if 'прогноз на' in x.lower()]].round(0) 

    df[[x for x in df if 'балл' in x.lower() or 'прогноз' in x.lower()]] = \
        df[[x for x in df if 'балл' in x.lower() or 'прогноз' in x.lower()]].fillna(0)

    df.fillna(value={
        'Название ЛПУ': '-',
        'Расстояние до ближайшего ЛПУ': 9999,
        'Территория': '-',
        'Число врачей в ближнем ЛПУ':0,
        'Число посещ. врачей в ближнем ЛПУ':0,
        'Число визитов в ближнее ЛПУ':0,
        "Число врачей в ЛПУ в радиусе 1 км":0,
        "Число посещ. врачей в радиусе 1 км":0,
         "Число визитов в ЛПУ в радиусе 1 км":0,
          'Продажи 28':0,
        'Продажи 56':0,
        'Месяц первых продаж':'-',
        "Число ЛПУ в радиусе 1 км":0,
        
        "Накопленный итог, Продажи #28":0,
        "Накопленный итог, Продажи #56":0,

        'Продажи 28(50)':0,
        'Продажи 28(100)':0,
        'Продажи 28(200)':0,

        'Продажи 56(50)':0,
        'Продажи 56(100)':0,
        'Продажи 56(200)':0,


        'Остатки 28':0,
        'Остатки 56':0,

        'Остатки 28(50)':0,
        'Остатки 28(100)':0,
        'Остатки 28(200)':0,

        'Остатки 56(50)':0,
        'Остатки 56(100)':0,
        'Остатки 56(200)':0,
        
        'Прогноз на следующий месяц 28(50)':0, 
        'Прогноз на пол года 28(50)':0, 
        'Прогноз на следующий месяц 28(100)':0, 
        'Прогноз на пол года 28(100)':0, 
        'Прогноз на следующий месяц 28(200)':0, 
        'Прогноз на пол года 28(200)':0, 
        'Прогноз на следующий месяц 28':0, 
        'Прогноз на полгода 28':0,
        
        'Прогноз на следующий месяц 56(50)':0, 
        'Прогноз на пол года 56(50)':0, 
        'Прогноз на следующий месяц 56(100)':0, 
        'Прогноз на пол года 56(100)':0, 
        'Прогноз на следующий месяц 56(200)':0, 
        'Прогноз на пол года 56(200)':0, 
        'Прогноз на следующий месяц 56':0, 
        'Прогноз на полгода 56':0
    }, inplace=True)

    df['Расстояние до ближнего ЛПУ'] = df['Расстояние до ближнего ЛПУ'].round(0) 

    for action_type in ('Продажи', 'Остатки'):
        df[f'Цвет_{action_type}'] = df[[x for x in df if action_type in x and 'месяц' not in x.lower()]].sum(1) >= 1
        df[f'Цвет_{action_type}'] = df[f'Цвет_{action_type}'].apply(lambda x: 'green' if x else 'red')

    df['Цвет'] = df.apply(stores_colors, axis=1)

    return df

In [10]:
l = []
for last_date in pd.date_range(start='2020-01-01', end='2021-06-01', freq='MS'):
    print(last_date)
    last_date = str(last_date.date())
    df1 = df_creator(last_date, future_date)
    l.append(df1)
df = pd.concat(l)

2020-01-01 00:00:00
2020-02-01 00:00:00
2020-03-01 00:00:00
2020-04-01 00:00:00
2020-05-01 00:00:00
2020-06-01 00:00:00
2020-07-01 00:00:00
2020-08-01 00:00:00
2020-09-01 00:00:00
2020-10-01 00:00:00
2020-11-01 00:00:00
2020-12-01 00:00:00
2021-01-01 00:00:00
2021-02-01 00:00:00
2021-03-01 00:00:00
2021-04-01 00:00:00
2021-05-01 00:00:00
2021-06-01 00:00:00


In [11]:
t = Table('apteki', db = db)

In [12]:
t.copy_df(df, 'replace')

In [11]:
regim = 'year'  #month or year
start='2021-07-01'
end='2021-12-01'
table='apteki'

In [12]:
def help_table(regim, start, end, db):
    if regim=='year':
        a = pd.date_range(start=start, end=end, freq='MS')
        l = []
        for date in a:
            l.append(str(date.date()))
    elif regim=='month':
        l=[start, end]
    else:
        print('Wrong regim')
        exit
    db.cur.execute('''drop table if exists julia_shaliapina.temporary_table;''')
    db.commit()
    db.cur.execute('''
    create table julia_shaliapina.temporary_table
    (date varchar(100))
    ''')
    db.commit()
    for date in l:
        db.cur.execute(f'''
        insert into julia_shaliapina.temporary_table
        values ('{date}')
        ''')
    db.commit()

In [13]:
def full_tab_creator(regim, start, end, df, table, db):
    help_table(regim, start, end, db)
    df1 = pd.read_sql(f'''
        with step1 as 
        (select distinct point_id, 
        "Адрес", "Регион",
               "Город", "Широта", "Долгота", "КАМ", "Аптечная сеть",
               "Месяц первых продаж",
               "Название ЛПУ", "Регион ЛПУ",
               "Город ЛПУ", "Адрес ЛПУ", "Широта ЛПУ", "Долгота ЛПУ",
               "Расстояние до ближнего ЛПУ", "Территория",
               "Флаг внимания", "Итоговый балл",
               "Квартиры, балл", "общественный транспорт, балл", "Трафик, балл",
               "Парковка, балл", "Детский сад, балл",
               "Прогноз на следующий месяц 28(50)", "Прогноз на пол года 28(50)",
               "Прогноз на следующий месяц 28(100)", "Прогноз на пол года 28(100)",
               "Прогноз на следующий месяц 28(200)", "Прогноз на пол года 28(200)",
               "Прогноз на следующий месяц 28", "Прогноз на полгода 28",
               "Прогноз на следующий месяц 56(50)", "Прогноз на пол года 56(50)",
               "Прогноз на следующий месяц 56(100)", "Прогноз на пол года 56(100)",
               "Прогноз на следующий месяц 56(200)", "Прогноз на пол года 56(200)",
               "Прогноз на следующий месяц 56", "Прогноз на полгода 56",
               "Цвет_Продажи", "Цвет_Остатки", "Цвет"
        from {table} )

        select*,
        date as "Дата",
        extract(year from date::date) as "Год", 
        concat('Q', extract(quarter from date::date),'`', to_char(date::date, 'YY')) "Квартал", 
        concat(extract(month from date::date), '`', to_char(date::date, 'YY')) as "Месяц"
        from step1 cross join julia_shaliapina.temporary_table

        ''', db.engine)
    df_full = df.append(df1, ignore_index=True)
    df_full.drop_duplicates(['point_id', 'Дата'], inplace=True)
    
    for typ in ("Прогноз на следующий месяц 28(50)", 
               "Прогноз на следующий месяц 28(100)",
               "Прогноз на следующий месяц 28(200)", 
               "Прогноз на следующий месяц 28", 
               "Прогноз на следующий месяц 56(50)", 
               "Прогноз на следующий месяц 56(100)", 
               "Прогноз на следующий месяц 56(200)", 
               "Прогноз на следующий месяц 56"):
        df_full.loc[df_full['Дата']!=start, typ]=np.nan
        
    for typ in ("Прогноз на пол года 28(50)",
                "Прогноз на пол года 28(100)",
               "Прогноз на пол года 28(200)",
               "Прогноз на полгода 28",
               "Прогноз на пол года 56(50)",
               "Прогноз на пол года 56(100)",
                "Прогноз на пол года 56(200)",
                "Прогноз на полгода 56"):
        df_full.loc[df_full['Дата']!=end, typ]=np.nan
        df_full.sort_values(by=['point_id', 'Дата'], inplace=True)
        df_full.reset_index(inplace=True, drop=True)
        
        
    return df_full

In [14]:
df_full = full_tab_creator(regim, start, end, df, table, db)

In [16]:
df_full.drop(columns='date', inplace=True, axis=1)

In [17]:
file_name ='apteki.xlsx'

writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
df_full.to_excel(writer, index = False, header=True, sheet_name='Куб', engine='xlsxwriter')

writer.book.use_zip64()

writer.save()